<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-12 10:10:31
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 67.67 K (0.51%)
Current PnL: -21.74 L (-14.79%)
CY Booked + Current PnL: -13.47 L (-9.16%)
-------------------
Total profit:  2.34 L
Total loss:  -24.08 L
-------------------
Total Booked + Current PnL: 13.51 L (11.25%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.21 L (61.0%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,3.93,218360.0,16880.0,10328.0,0.07,8.38,4.73,13.51,61.0,1.63,1.67,22.03,XY25,NTT,BANKS
55,MEDANTA,1087.93,1486.00,12.48,H-MC,13.95,160602.0,34402.0,11772.0,-0.21,27.26,7.33,36.59,111.0,2.92,1.23,43.19,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.72,107656.0,3344.0,13306.0,0.01,3.21,12.36,15.96,11.0,0.25,0.82,20.65,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,93.17,M-SC,10.73,83231.0,-17546.0,17637.0,-0.13,-17.41,21.19,0.09,243.0,-0.99,0.64,9.35,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.26,H-SC,9.13,220259.0,27239.0,20792.0,0.62,14.11,9.44,24.89,134.0,1.31,1.69,40.75,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-8.88,H-MC,4.04,198276.0,1248.0,88372.0,-0.32,0.63,44.57,45.49,103.0,0.01,1.52,13.76,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-30.94,H-MC,6.40,199950.0,-870.0,70982.0,0.21,-0.43,35.50,34.92,82.0,-0.01,1.53,4.22,X40,ATH,FMCG
56,NATIONALUM,189.63,247.44,-50.29,H-MC,2.11,99640.0,-864.0,31506.0,0.50,-0.86,31.62,30.49,80.0,-0.03,0.76,31.27,MH,ATH,METALS
92,WIPRO,243.46,420.00,-13.34,M-LC,6.79,152353.0,1408.0,108049.0,1.70,0.93,70.92,72.51,54.0,0.01,1.17,7.01,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.18,H-LC,2.00,207350.0,2850.0,38567.0,0.58,1.39,18.60,20.25,10.0,0.07,1.59,10.25,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.80,H-LC,2.56,170000.0,-31386.0,119000.0,0.44,-15.59,70.00,43.51,27.0,-0.26,1.30,18.34,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.65,H-LC,3.60,184932.0,-15378.0,75360.0,1.39,-7.68,40.75,29.94,17.0,-0.20,1.42,27.76,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-15.86,H-LC,3.80,129437.0,-135989.0,192900.0,0.12,-51.23,149.03,21.44,8.0,-0.70,0.99,4.45,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-20.17,H-LC,5.40,159555.0,3378.0,22721.0,-0.09,2.16,14.24,16.71,19.0,0.15,1.22,19.82,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.4,-2.50,M-SC,23.32,93872.0,-82943.0,163741.0,9.19,-46.91,174.43,45.70,197.0,-0.51,0.72,9.19,XY24,NTT,TRAVEL
81,TITAGARH,1097.23,1548.0,-13.27,H-SC,7.36,142135.0,-49880.0,128760.0,4.64,-25.98,90.59,41.08,156.0,-0.39,1.09,20.30,XY24,NTT,ENGINEERING
66,SAMMAANCAP,170.35,326.0,-161.36,M-SC,4.70,70740.0,-31470.0,124863.0,2.55,-30.79,176.51,91.37,206.0,-0.25,0.54,14.83,XY25,NTT,FINANCE
6,ASIANTILES,75.41,137.0,6622.22,L-SC,6.79,73757.0,-20053.0,96673.0,2.08,-21.38,131.07,81.67,270.0,-0.21,0.56,42.83,XR,NTT,CERAMICS
92,WIPRO,243.46,420.0,-13.34,M-LC,6.79,152353.0,1408.0,108049.0,1.70,0.93,70.92,72.51,54.0,0.01,1.17,7.01,XR,NTT,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.00,-35.44,M-SC,6.09,96595.0,-32075.0,77373.0,-1.66,-24.93,80.10,35.20,209.0,-0.41,0.74,0.00,X40,NTT,FOOTWEAR
27,DIXON,14367.00,18940.15,-36.39,H-MC,6.48,127344.0,12408.0,24183.0,-1.06,10.80,18.99,31.83,83.0,0.51,0.97,36.52,X40N,ATH,IT
15,BLUESTARCO,1646.70,2326.38,1.21,H-MC,7.43,174730.0,10060.0,57906.0,-0.83,6.11,33.14,41.28,91.0,0.17,1.34,14.58,X40N,ATH,AC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.31,69427.0,-44122.0,84173.0,-0.80,-38.86,121.24,35.27,268.0,-0.52,0.53,69.92,XR,NTT,HOTELS
87,VAIBHAVGBL,350.21,521.00,42.70,H-SC,8.21,95289.0,-56702.0,130822.0,-0.73,-37.31,137.29,48.77,125.0,-0.43,0.73,10.14,XR,NTT,JEWELLERY


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.17,M-SC,10.73,83231.0,-17546.0,17637.0,-0.13,-17.41,21.19,0.09,243.0,-0.99,0.64,9.35,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,20.59,117423.0,-25056.0,44574.0,1.64,-17.59,37.96,13.70,163.0,-0.56,0.90,38.59,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.09,H-SC,3.12,115416.0,-28387.0,55134.0,0.38,-19.74,47.77,18.60,148.0,-0.51,0.88,27.37,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-69.25,H-SC,2.76,212454.0,-57213.0,93543.0,0.44,-21.22,44.03,13.47,142.0,-0.61,1.63,7.46,XY24,NTT,PAINTS
78,TATAELXSI,7332.28,9161.0,-19.68,H-MC,2.70,80052.0,-22600.0,48199.0,0.73,-22.02,60.21,24.94,99.0,-0.47,0.61,21.69,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-27.40,H-LC,15.62,240575.0,-55359.0,149277.0,1.43,-18.71,62.05,31.73,2.0,-0.37,1.84,2.71,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,10.15,257527.0,-4074.0,149700.0,0.74,-1.56,58.13,55.67,4.0,-0.03,1.97,4.45,X40,BTT,IT
89,VBL,492.64,671.64,-8.60,H-LC,10.59,277635.0,12595.0,83707.0,1.04,4.75,30.15,36.33,5.0,0.15,2.13,18.68,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,9.98,197320.0,-19199.0,112097.0,0.25,-8.87,56.81,42.91,7.0,-0.17,1.51,3.34,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-15.86,H-LC,3.80,129437.0,-135989.0,192900.0,0.12,-51.23,149.03,21.44,8.0,-0.70,0.99,4.45,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.48,H-MC,13.95,160602.0,34402.0,11772.0,-0.21,27.26,7.33,36.59,111.0,2.92,1.23,43.19,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.48,H-MC,13.95,160602.0,34402.0,11772.0,-0.21,27.26,7.33,36.59,111.0,2.92,1.23,43.19,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,11.32,H-SC,12.97,132552.0,7255.0,51748.0,0.10,5.79,39.04,47.09,141.0,0.14,1.01,65.90,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.21,H-MC,7.43,174730.0,10060.0,57906.0,-0.83,6.11,33.14,41.28,91.0,0.17,1.34,14.58,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-6.27,H-SC,3.05,106509.0,4505.0,39419.0,0.15,4.42,37.01,43.06,123.0,0.11,0.82,22.29,X40N,ATH,MISC
25,DABUR,505.20,735.00,-8.88,H-MC,4.04,198276.0,1248.0,88372.0,-0.32,0.63,44.57,45.49,103.0,0.01,1.52,13.76,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-12.51,H-MC,6.89,162176.0,11263.0,46042.0,1.57,7.46,28.39,37.97,92.0,0.24,1.24,34.66,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,11.32,H-SC,12.97,132552.0,7255.0,51748.0,0.10,5.79,39.04,47.09,141.0,0.14,1.01,65.90,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.28,H-SC,12.54,136104.0,12768.0,120996.0,1.08,10.35,88.90,108.45,119.0,0.11,1.04,35.02,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-44.20,M-SC,4.09,94380.0,2882.0,75872.0,-0.29,3.15,80.39,86.07,219.0,0.04,0.72,36.53,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-13.34,M-LC,6.79,152353.0,1408.0,108049.0,1.70,0.93,70.92,72.51,54.0,0.01,1.17,7.01,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.31,69427.0,-44122.0,84173.0,-0.80,-38.86,121.24,35.27,268.0,-0.52,0.53,69.92,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.50,H-SC,16.85,89767.0,-11196.0,105728.0,0.29,-11.09,117.78,93.63,149.0,-0.11,0.69,33.71,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.64,90900.0,-7080.0,28952.0,0.68,-7.23,31.85,22.32,152.0,-0.24,0.70,31.84,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,20.59,117423.0,-25056.0,44574.0,1.64,-17.59,37.96,13.70,163.0,-0.56,0.90,38.59,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,11.32,H-SC,12.97,132552.0,7255.0,51748.0,0.10,5.79,39.04,47.09,141.0,0.14,1.01,65.90,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-8.60,H-LC,10.59,277635.0,12595.0,83707.0,1.04,4.75,30.15,36.33,5.0,0.15,2.13,18.68,X40N,ATH,FMCG
51,LALPATHLAB,2838.53,3545.00,-26.26,H-SC,9.13,220259.0,27239.0,20792.0,0.62,14.11,9.44,24.89,134.0,1.31,1.69,40.75,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-12.51,H-MC,6.89,162176.0,11263.0,46042.0,1.57,7.46,28.39,37.97,92.0,0.24,1.24,34.66,AR,ATH,AUTO
70,SHALBY,261.39,327.00,928.71,M-SC,7.39,139993.0,-41150.0,86614.0,-0.40,-22.72,61.87,25.10,232.0,-0.48,1.07,12.19,XY24,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.31,69427.0,-44122.0,84173.0,-0.80,-38.86,121.24,35.27,268.0,-0.52,0.53,69.92,XR,NTT,HOTELS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.51
1,25,41.15
2,50,70.90


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.53
LC    34.17
MC    26.30
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.05
X40N     15.33
X40      13.42
XY25     10.69
XR        8.98
AR        8.42
OX40N     7.56
X5K       2.23
MH        1.77
X200      1.42
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.83
H-SC    24.61
H-MC    23.35
M-SC    13.47
M-LC     7.25
M-MC     2.61
L-SC     1.45
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.33,-10.19,45.61
IT,12.05,-18.55,77.76
FINANCE,8.34,-19.55,63.78
BANKS,7.65,-14.06,62.49
MISC,5.93,-18.78,86.86
PAINTS,5.66,-18.12,41.39
HEALTHCARE,5.21,3.51,20.85
ELECTRICAL,4.70,-14.56,64.18
AUTO,4.43,-16.55,67.26


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2842085.0,23
XR,1120850.0,13
AR,1107934.0,10
X40,818831.0,10
XY25,652720.0,8
X40N,606841.0,12
OX40N,470677.0,10
SR,195865.0,2
X5K,147033.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2804148.0,28
M-SC,1486158.0,17
H-LC,1362815.0,18
H-MC,1326798.0,18
M-LC,431892.0,5
M-MC,336160.0,2
L-SC,270679.0,3
L-MC,57818.0,1
L-LC,44982.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      936083.0      6
M-SC       XY24      865136.0      7
H-SC       AR        668848.0      5
           XR        530588.0      6
H-LC       X40       511681.0      5
H-MC       XY24      485397.0      5
M-MC       XY24      336160.0      2
H-LC       XY25      250074.0      3
           X40N      239893.0      5
           AR        232980.0      2
H-MC       X40       229777.0      4
           XY25      222473.0      2
H-SC       X40N      221553.0      4
           OX40N     199463.0      4
           SR        195865.0      2
L-SC       XR        180846.0      2
M-LC       XY24      179788.0      2
M-SC       AR        160064.0      2
H-MC       X40N      145395.0      3
M-LC       X5K       133727.0      1
M-SC       OX40N     133182.0      4
           XR        125540.0      2
           XY25      124863.0      1
H-MC       XR        118009.0      1
M-LC       XR        108049.0      1
L-SC       OX40N      89833.0      1
M-SC       X40        77373.0      1
H-LC       X200       75360.0      1
L-MC       XR         57818.0      1
H-SC       MH         51748.0      1
H-MC       OX40N      48199.0      1
           AR         46042.0      1
L-LC       XY25       44982.0      1
H-LC       XY24       39521.0      1
H-MC       MH         31506.0      1
H-LC       X5K        13306.0      1
M-LC       XY25       10328.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
